# Task 3
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [5]:
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import os
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
# Add any other imports you need here
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# The device is automatically set to GPU if available, otherwise CPU
# If you want to force the device to CPU, you can change the line to
# device = torch.device("cpu")
# When using the GPU, it is important that your model and all data are on the
# same device.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
"""
Transform, resize and normalize the images and then use a pretrained model to extract
the embeddings.
"""
# TODO: define a transform to pre-process the images
# The required pre-processing depends on the pre-trained model you choose
# below.
# See https://pytorch.org/vision/stable/models.html#using-the-pre-trained-models
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to (224, 224)
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/dataset/", transform=train_transforms)
# Hint: adjust batch_size and num_workers to your PC configuration, so that you don't
# run out of memory (VRAM if on GPU, RAM if on CPU)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=64,
                          shuffle=False,
                          pin_memory=True, num_workers=16)

# TODO: define a model for extraction of the embeddings (Hint: load a pretrained model,
# more info here: https://pytorch.org/vision/stable/models.html)
model = models.resnet18(pretrained=True)

embedding_size = 512  # Replace with the actual embedding size
model.fc = nn.Linear(model.fc.in_features, embedding_size)
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# pick your model
num_images = len(train_dataset)
embeddings = np.zeros((num_images, embedding_size))

# TODO: Use the model to extract the embeddings. Hint: remove the last layers of the
# model to access the embeddings the model generates.
# Loop through the dataset and extract embeddings
for i, (image, _) in enumerate(train_loader):
    image = image.to(device)
    with torch.no_grad():
        features = model(image)
    embeddings[i * train_loader.batch_size: (i + 1) * train_loader.batch_size] = features.cpu().numpy()

np.save('/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/dataset/embeddings.npy', embeddings)

In [9]:
def get_data(file, train=True):
    """
    Load the triplets from the file and generate the features and labels.

    input: file: string, the path to the file containing the triplets
          train: boolean, whether the data is for training or testing

    output: X: numpy array, the features
            y: numpy array, the labels
    """
    triplets = []
    with open(file) as f:
        for line in f:
            triplets.append(line)

    # generate training data from triplets
    train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/dataset/",
                                         transform=None)
    filenames = [s[0].split('/')[-1].replace('.jpg', '') for s in train_dataset.samples]
    embeddings = np.load('/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/dataset/embeddings.npy')
    # TODO: Normalize the embeddings
    embeddings /= np.linalg.norm(embeddings, axis=1, keepdims=True)

    file_to_embedding = {filename: emb for filename, emb in zip(filenames, embeddings)}
    #for i in range(len(filenames)):
        #file_to_embedding[filenames[i]] = embeddings[i]
    X = []
    y = []
    for t in triplets:
        emb = [file_to_embedding[a] for a in t.split()]
        X.append(np.hstack([emb[0], emb[1], emb[2]]))
        y.append(1)
        if train:
            X.append(np.hstack([emb[0], emb[2], emb[1]]))
            y.append(0)
    X = np.vstack(X)
    y = np.hstack(y)
    return X, y

Hint: adjust batch_size and num_workers to your PC configuration, so that you don't run out of memory (VRAM if on GPU, RAM if on CPU)

In [10]:
def create_loader_from_np(X, y=None, train=True, batch_size=64, shuffle=True, num_workers=4):
    if train:
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float),
                                torch.from_numpy(y).type(torch.long))
    else:
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float))
    loader = DataLoader(dataset=dataset,
                        batch_size=batch_size,
                        shuffle=shuffle,
                        pin_memory=True, num_workers=num_workers)
    return loader

TODO: define a model. Here, the basic structure is defined, but you need to fill in the details

In [11]:
class Net(nn.Module):
    """
    The model class, which defines our classifier.
    """
    def __init__(self, input_size):
        """
        The constructor of the model.
        """
        super().__init__()
        self.fc = nn.Linear(input_size, 1)

    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        x = self.fc(x)
        #x = F.relu(x)
        x = torch.sigmoid(x)
        return x

In [12]:
TRAIN_TRIPLETS = '/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/train_triplets.txt'

# load the training data
X, y = get_data(TRAIN_TRIPLETS)
# Create data loaders for the training data
train_loader = create_loader_from_np(X, y, train = True, batch_size=64)
# delete the loaded training data to save memory, as the data loader copies
del X
del y

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [13]:
TEST_TRIPLETS = '/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/test_triplets.txt'

# repeat for testing data
X_test, y_test = get_data(TEST_TRIPLETS, train=False)
test_loader = create_loader_from_np(X_test, train = False, batch_size=2048, shuffle=False)
del X_test
del y_test

In [16]:
"""
The training procedure of the model; it accepts the training data, defines the model
and then trains it.

input: train_loader: torch.data.util.DataLoader, the object containing the training data

compute: model: torch.nn.Module, the trained model
"""
model = Net(3*512)
#model.train()
model.to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
n_epochs = 10
# TODO: define a loss function, optimizer and proceed with training. Hint: use the part
# of the training data as a validation split. After each epoch, compute the loss on the
# validation split and print it out. This enables you to see how your model is performing
# on the validation data before submitting the results on the server. After choosing the
# best model, train it on the whole training data.
for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss:.4f}")

Epoch 1/10, Loss: 0.6849
Epoch 2/10, Loss: 0.6800
Epoch 3/10, Loss: 0.6783
Epoch 4/10, Loss: 0.6773
Epoch 5/10, Loss: 0.6765
Epoch 6/10, Loss: 0.6760
Epoch 7/10, Loss: 0.6756
Epoch 8/10, Loss: 0.6751
Epoch 9/10, Loss: 0.6748
Epoch 10/10, Loss: 0.6746


In [17]:
"""
The testing procedure of the model; it accepts the testing data and the trained model and
then tests the model on it.

input: model: torch.nn.Module, the trained model
       loader: torch.data.util.DataLoader, the object containing the testing data

computea: None, the function saves the predictions to a results.txt file
"""
model.eval()
predictions = []
# Iterate over the test data
with torch.no_grad(): # We don't need to compute gradients for testing
    for [x_batch] in test_loader:
        x_batch= x_batch.to(device)
        predicted = model(x_batch)
        predicted = predicted.cpu().numpy()
        # Rounding the predictions to 0 or 1
        predicted[predicted >= 0.5] = 1
        predicted[predicted < 0.5] = 0
        predictions.append(predicted)
    predictions = np.vstack(predictions)
np.savetxt("/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/results_1.txt", predictions, fmt='%i')